In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date
import os
import os, glob

In [3]:
path = r'C:\path_to_data' ## Insert your directory
os.chdir(path)

# Define hydrological drought extraction function

In [5]:
def drough_extract(id, th_type, moving_window, thresh_value, min_dur, path_id, path_out):
# (th_type = threshold type, that can be fixed or variable; moving_window in number of days; thresh_value in flow percentile; min_dur in days)
# the streamflow values has to be in mm/day    

    moving_window=moving_window
    thresh_value=thresh_value
    min_dur=min_dur

  ###  load df
    df = pd.read_csv(path_id)

  ### convert date to posixct format
    df = df.dropna()
    df['Datetime']=pd.to_datetime(df['Datetime'], format="%Y-%m-%d", errors="coerce").fillna(
        pd.to_datetime(df['Datetime'], format= "%m/%d/%Y", errors="coerce"))

    

  ### extract date details
    df["day"] = df['Datetime'].map(lambda x: x.day)
    df["month"] = df['Datetime'].map(lambda x: x.month)
    df["year"] = df['Datetime'].map(lambda x: x.year)

  ### remove 29.02
    drop=list(df[(df['day'] ==29) & (df['month'] ==2)].index)
    df=df.drop(drop)
    df=df.reset_index(drop=True)

  ### Smooth time series
    ts=df['streamflow(mm/d)']
    
    ma_ts=ts.rolling(moving_window, min_periods=1, center=True).mean()

  #### fixed threshold
    if th_type == 'fixed':
        thr = np.nanquantile(df['streamflow(mm/d)'],thresh_value)
        thr=np.full(365, thr)
      ### in case of a zero threshold, slightly elevate the threshold
        if thr[0]==0:
            thr=thr.np.full(0.01,365)
        else:
            thr=np.full(365, thr)
    converter= lambda x: x.timetuple().tm_yday
    
 #### variable threshold   
    if th_type == 'variable':
        if (len(df.year))==len(df ['year'])*365: 
            df['day_id']==np.tile( np.arange(0,365),(len(df.year.unique())))
        else:
            df['day_id']=df['Datetime'].map(converter)

        thr=[]
        
        for d in range(0,365):
        ### define window length
            win_length=np.arange(0,15) 
            before = list(df.day_id[d+365-win_length])
            after = list(df.day_id[d+365+win_length-1])
            ### define days within window
            ids_win = before+after
            ### determine values in window around day i
            data_window = ma_ts[df[df['day_id'].isin(ids_win)].index]
            quant_value = np.nanquantile(data_window,thresh_value)
            thr.append(quant_value)
            
    ### create column in table with corresponding thresholds
    df['threshold']=np.tile(thr,len(df.year.unique()))[:df.shape[0]]

    ### idetify events below threshold
    deficit_ids=np.where(ma_ts < df['threshold'])[0].tolist()
    deficit_datetime=df[df.index.isin(np.where(ma_ts < df['threshold'])[0].tolist())].Datetime
    end=np.where(np.array([x - deficit_ids[i - 1] for i, x in enumerate(deficit_ids)][1:]) >1)[0]
    start=end+1
    start=np.insert(start, 0,0)
    end=np.insert(end,len(end),(len(deficit_ids)-1))
    start_ids=[deficit_ids[index] for index in start]
    end_ids=[deficit_ids[index] for index in end]

   ### compute drought characteristics
    duration = []
    deficit = []
    intensity = []
    start_date = []
    end_date = []
    date = []
    thresh_events = []
    
    for e in range(len(start_ids)):
        duration.append(end[e] - start[e])
        deficit.append(abs(sum(ma_ts[start_ids[e]:(end_ids[e]+1)]-df.threshold[start_ids[e]:(end_ids[e]+1)])))
        intensity.append(min(ma_ts[start_ids[e]:(end_ids[e]+1)]))
        start_date.append(df.Datetime[start_ids[e]])
        end_date.append(df.Datetime[end_ids[e]])
        date.append(df[start_ids[e]:(end_ids[e]+1)].loc[ma_ts[start_ids[e]:(end_ids[e]+1)].idxmin()].Datetime)
        thresh_events.append(df[start_ids[e]:(end_ids[e]+1)].loc[ma_ts[start_ids[e]:(end_ids[e]+1)].idxmin()].threshold)

    event_df=pd.DataFrame(
        {'start_date':start_date,
        'end_date':end_date,
        'date':date,
        'thresh_events':thresh_events,
         'duration': duration, #days
        'deficit': deficit, #mm
        'intensity': intensity, #mm/day
        })
  ### remove events shorter than the minimum duration
    if len(np.where(event_df.duration<min_dur)[0].tolist()) >0:
        event_df=event_df.drop(np.where(event_df.duration<min_dur)[0].tolist(),axis=0) 
        
  ### save files
    path_to_save=os.path.join(path_out,"Cabra_"+str(id)+'_th'+str(thresh_value)+'_mw'+str(moving_window)+'_md'+str(min_dur))
    event_df.to_csv(path_to_save+".csv", sep=',',index=False)

## load data and apply

In [7]:
##Load data
path_id = os.listdir('database_mm/')
path_out= ''
path_att = pd.read_csv('CABra_attibutes.csv', sep=',', index_col= 'id')

In [9]:
for i in path_id:
    id=int(i.split("_")[1])
    path=os.path.join('database_mm/',i)
    zeros= np.count_nonzero(pd.read_csv(path)['streamflow(mm/d)']==0)
    if float(zeros)<=0.05:
        drough_extract(id, 'variable', 30, 0.15, 30, path, path_out)
        print("successfully done:",i)
    else:
        print(i, "is ephemeral") 

successfully done: CABra_100_mmd.csv
successfully done: CABra_101_mmd.csv
successfully done: CABra_102_mmd.csv
successfully done: CABra_105_mmd.csv
successfully done: CABra_106_mmd.csv
CABra_107_mmd.csv is ephemeral
successfully done: CABra_108_mmd.csv
CABra_109_mmd.csv is ephemeral
successfully done: CABra_10_mmd.csv
successfully done: CABra_110_mmd.csv
successfully done: CABra_111_mmd.csv
successfully done: CABra_112_mmd.csv
successfully done: CABra_113_mmd.csv
successfully done: CABra_114_mmd.csv
successfully done: CABra_115_mmd.csv
successfully done: CABra_116_mmd.csv
successfully done: CABra_117_mmd.csv
successfully done: CABra_118_mmd.csv
successfully done: CABra_119_mmd.csv
successfully done: CABra_11_mmd.csv
successfully done: CABra_122_mmd.csv
successfully done: CABra_123_mmd.csv
successfully done: CABra_124_mmd.csv
successfully done: CABra_125_mmd.csv
successfully done: CABra_126_mmd.csv
successfully done: CABra_127_mmd.csv
CABra_128_mmd.csv is ephemeral
CABra_129_mmd.csv is 